In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df0 = pd.read_csv('expense.csv', index_col = 0)

In [3]:
expense_collect= dict.fromkeys(['Automobile - Gasoline', 'Automobile - Maintenance', 'Automobile - Payment', 
                                'Insurance - Auto'], 'Automobile')
expense_collect.update(dict.fromkeys(['Taxes - Federal', 'Taxes - State', 'Taxes - Other Taxes'], 'Income Tax'))
expense_collect.update(dict.fromkeys(['Taxes - Real Estate', 'Household - Laundry/Dry Cleaning', 'Rent', 'Mortgage',
                                      'Household - Maintenance/Repairs', 'Household - Other Household', 'Insurance - Home/Renter', 
                                      'Fees - Realtor/Appraiser'], 'Household'))
expense_collect.update(dict.fromkeys(['Personal Needs', 'Miscellaneous Expense'], 'Personal Misc'))
expense_collect.update(dict.fromkeys(['Utilities - Telephone/Internet/Cable', 'Utilities - Garbage', 
                                      'Utilities - Gas & Electric', 'Utilities - Water/Sewer', 
                                      'Subscriptions & Dues'], 'Utilities'))
expense_collect.update(dict.fromkeys(['Hobby', 'Entertainment', 'Pet Care'], 'Hobbey'))
expense_collect.update(dict.fromkeys(['Insurance - Medical', 'Medical - Ambulance/Transport', 
                                      'Medical - Dental', 'Medical - Doctor/Hospital', 'Medical: Equipment', 
                                      'Medical - Prescriptions'], 'Medical'))
expense_collect.update(dict.fromkeys(['Transportation', 'Travel'], 'Travel'))
expense_collect.update(dict.fromkeys(['Fees - Court', 'Fees - Attorney'], 'Court'))
expense_collect.update(dict.fromkeys(['Fees - Accountant', 'Bank Service Charges', 'Bond Premium', 
                                      'Debt - Interest', 'Fees - Late'], 'Banking'))
expense_collect.update(dict.fromkeys(['Services - Cleaning', 'Services - Personal Care'], 'Services'))
expense_collect.update(dict.fromkeys(['Food - Groceries', 'Food - Dining Out'], 'Food'))
expense_collect.update(dict.fromkeys(['Charitable Donation','Gifts Given'], 'Charity Gifts'))

expense_collect.update(dict.fromkeys([]))
expense_collect['Care Facility'] = 'Care Facility'
expense_collect['Fees - Guardian/Conservator'] = 'Fees - Guardian_Conservator'

expense_collect['Insurance - Other Insurance'] = 'Insurance - Other Insurance'
expense_collect['Clothing'] = 'Clothing'
expense_collect['Fees - Other Fees'] = 'Fees - Other Fees'
expense_collect['Asset Depreciation'] = 'Asset Depreciation'
expense_collect['Personal Property - Purchase'] = 'Personal Property - Purchase'
expense_collect['Support Payments'] = 'Support Payments'
expense_collect['Funeral'] = 'Funeral'
expense_collect['Education'] = 'Education'

In [4]:
df0['TranCatColl'] = df0['TranDescr'].map(expense_collect)
df0.head()

,IncExpId,CasFil,Expense,Amount,TranDt,TranCat,TranDescr,CourtOrdTran,CarFac,NumDays,TranCatColl
0,4673,4815,1,1250.0,2014-04-15,185,Care Facility,0,1,1562.0,Care Facility
1,4674,4815,1,104.0,2014-04-15,214,Personal Needs,0,0,1562.0,Personal Misc
2,4675,4815,1,352.0,2014-04-15,240,Medical - Prescriptions,0,0,1562.0,Medical
3,4676,4816,1,500.0,2014-04-15,214,Personal Needs,0,0,1562.0,Personal Misc
4,4677,4820,1,456.0,2014-04-15,239,Rent,0,0,1562.0,Household


In [5]:
cat = 'Personal Misc'

def counting_neighbours(cat):
    average = df0[df0['TranCatColl']==cat].groupby('CasFil')['Amount'].mean()
    count = df0[df0['TranCatColl']==cat].groupby('CasFil')['Amount'].count()
    dura = df0[df0['TranCatColl']==cat].groupby('CasFil')['NumDays'].mean()
    count = np.array(count/dura)
    casfil = np.array(dura.index)
    aver = np.array(average.values)
    norm_aver = aver * np.max(count) / np.max(aver)

    arr = np.stack((casfil, count, norm_aver), axis=1)

    dist = []
    length = len(arr)
    for i in xrange(length):
        for j in xrange(i+1, length):
            dist.append(np.linalg.norm(arr[i, 1:] - arr[j, 1:]))

    med = np.median(dist)

    neighbours = []
    for i in xrange(length):
        dist = []
        for j in xrange(length):
            if i != j:
                dist.append(np.linalg.norm(arr[i, 1:] - arr[j, 1:]))
        dist = np.array(dist)
        neighbours.append(np.sum(dist < med))
    neighbours = np.array(neighbours)

    arr = np.stack((casfil, count, aver, neighbours), axis =1)

    filename = cat + str('.csv')
    np.savetxt(filename, arr, delimiter=',')

In [10]:
cat = 'Personal Misc'
counting_neighbours(cat)


In [6]:
collected_categories = df0['TranCatColl'].unique()

In [8]:
collected_categories

array(['Care Facility', 'Personal Misc', 'Medical', 'Household',
       'Income Tax', 'Fees - Guardian/Conservator', 'Banking', 'Utilities',
       'Insurance - Other Insurance', 'Court', 'Travel', 'Food', 'Hobbey',
       'Clothing', 'Charity Gifts', 'Automobile', 'Services',
       'Fees - Other Fees', 'Asset Depreciation',
       'Personal Property - Purchase', 'Education', 'Support Payments',
       'Funeral'], dtype=object)

In [9]:
categories = ['Banking', 'Utilities',
       'Insurance - Other Insurance', 'Court', 'Travel', 'Food', 'Hobbey',
       'Clothing', 'Charity Gifts', 'Automobile', 'Services',
       'Fees - Other Fees', 'Asset Depreciation',
       'Personal Property - Purchase', 'Education', 'Support Payments',
       'Funeral']

In [10]:
for cat in categories:
    counting_neighbours(cat)

Error with 'Fees - Guardian/Conservator'

In [11]:
cat = 'Fees - Guardian_Conservator'
average = df0[df0['TranCatColl']==cat].groupby('CasFil')['Amount'].mean()
count = df0[df0['TranCatColl']==cat].groupby('CasFil')['Amount'].count()
dura = df0[df0['TranCatColl']==cat].groupby('CasFil')['NumDays'].mean()
count = np.array(count/dura)
casfil = np.array(dura.index)
aver = np.array(average.values)
norm_aver = aver * np.max(count) / np.max(aver)

arr = np.stack((casfil, count, norm_aver), axis=1)

dist = []
length = len(arr)
for i in xrange(length):
    for j in xrange(i+1, length):
        dist.append(np.linalg.norm(arr[i, 1:] - arr[j, 1:]))

med = np.median(dist)

neighbours = []
for i in xrange(length):
    dist = []
    for j in xrange(length):
        if i != j:
            dist.append(np.linalg.norm(arr[i, 1:] - arr[j, 1:]))
    dist = np.array(dist)
    neighbours.append(np.sum(dist < med))
neighbours = np.array(neighbours)

arr = np.stack((casfil, count, aver, neighbours), axis =1)

filename = 'fees_guardian_conservator.csv'
np.savetxt(filename, arr, delimiter=',')